In [4]:
# Imports
import rasterio
from sklearn.cluster import DBSCAN
from pyproj import transform
from rasterio.crs import CRS
from rasterio.warp import transform

FILEPATH_PREFIX = "downloads/S2A_MSIL2A_20210716T095031_N0301_R079_T34UEF_20210716T120258.SAFE/GRANULE/L2A_T34UEF_A031679_20210716T095359/IMG_DATA/R10m/T34UEF_20210716T095031"
CLUSTER_THRESHOLD = 8000

In [5]:
# Read data
dataset_band_1 = rasterio.open(FILEPATH_PREFIX + '_B03_10m.jp2')
dataset_band_2 = rasterio.open(FILEPATH_PREFIX + '_B08_10m.jp2')

In [6]:
import numpy as np

# Water Mask calculation
image_band_1 = dataset_band_1.read(1)
image_band_2 = dataset_band_2.read(1)
image_band_1_norm = image_band_1 / np.max(np.abs(image_band_1))
image_band_2_norm = image_band_2 / np.max(np.abs(image_band_2))
image_ndwi = (image_band_1_norm - image_band_2_norm) // (image_band_1_norm + image_band_2_norm + np.ones((image_band_1_norm.shape[0], image_band_1_norm.shape[1]))) + np.ones((image_band_1_norm.shape[0], image_band_1_norm.shape[1]))
water_mask = image_ndwi > 0.1

In [7]:
# Clustering
water_indexes = np.transpose(water_mask.nonzero())
clusters = DBSCAN(eps = 5.0, min_samples = 10, algorithm='kd_tree', n_jobs = -1).fit(water_indexes)
unique, counts = np.unique(clusters.labels_, return_counts = True)
cluster_indexes = dict(zip(unique, counts))
cluster_indexes_above_thre = {k: v for k, v in cluster_indexes.items() if v > CLUSTER_THRESHOLD and k != -1}
cluster_mask = [idx in cluster_indexes_above_thre for idx in clusters.labels_]
water_indexes_image_coords = water_indexes[cluster_mask]


In [8]:
# Real Coords calculation
data_transform = dataset_band_1.transform
move_to_real_coords = lambda water_idx: data_transform * water_idx

zipped_water_clusters = zip(water_indexes, clusters.labels_)
water_cluster_points = {}
for point, cluster_idx in zipped_water_clusters:
    if cluster_idx not in water_cluster_points and cluster_idx in cluster_indexes_above_thre:
        water_cluster_points[cluster_idx] = point

water_indexes_real_coords = np.array([move_to_real_coords(np.array([xi[1], xi[0]])) for xi in water_cluster_points.values()])

zipped_final_result = zip(water_cluster_points, water_indexes_real_coords)
new_crs = CRS.from_epsg(4326)

export_to_db_dic = {}

for result in list(zipped_final_result):
    new_coo = transform(dataset_band_2.crs, new_crs, xs=[result[1][0]], ys=[result[1][1]])
    export_to_db_dic[result[0]] = [new_coo[1][0], new_coo[0][0]]

export_to_db_dic


{0: [55.04698603153837, 20.999686980376268],
 54: [54.2472672223876, 21.58271842653359],
 59: [54.1983677088979, 21.84216306885771],
 64: [54.19266073865039, 21.75391702182309],
 72: [54.18397463696728, 21.52834905566329],
 85: [54.15908090195105, 21.7626483874136],
 99: [54.14705799974809, 21.730276297258655],
 108: [54.12909932825082, 21.947117652384584],
 124: [54.119998050119435, 21.669518372196475],
 134: [54.114164996823376, 21.772991169832792],
 148: [54.104409713653844, 22.19631851699858],
 195: [54.06664345066743, 21.922469135107754]}